In [2]:
import os
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from data_utils import *

#### Загружаем отобранные тэги

In [ ]:
tags = pd.read_csv("./data/tags/mk_tags_freq.csv", index_col=0)
tags.drop(tags.iloc[:2].index, inplace=True)
tags.reset_index(inplace=True, drop=True)
path = './data/tags/posts_mk/'

In [ ]:
tags.head()

In [4]:
tags.shape

(124, 2)

# TODO: Переписать функции в модуль

Собираем данные в csv

In [ ]:
def harvest_tags(tags, path):
    """Dumps data searched by instagram tags to path"""
    
    for index, row in tags.iterrows():
        tag = row["tag"]
        prefix = f"{tag}_{index}"
        gt.get_tag_data(tag, dump=True, path=path, prefix=prefix)

Сливаем файлы в одну таблицу

In [9]:
def merge_from_path(path):
    """Merges all csv tables (csv batches) from path into one pandas.DataFrame"""
    table = []
    
    for file_name in next(os.walk(path))[2]:
        next_table = pd.read_csv(os.path.join(path, file_name), sep=";", index_col=0, engine="python", encoding="utf-8")
        next_table["by_tag"] = file_name.split("_")[0]
        print(f"Next table shape: {next_table.shape}")
        table.append(next_table)
#         table = pd.concat([table, next_table], sort=False)

#     data = table.reset_index().drop(columns=["index"]).drop_duplicates(subset="post_id")
    return pd.concat(table, sort=False)

In [10]:
merge_from_path("./data/posts")

Next table shape: (191, 7)
Next table shape: (92, 7)
Next table shape: (622, 7)
Next table shape: (21250, 7)
Next table shape: (12150, 7)
Next table shape: (19287, 7)
Next table shape: (7686, 7)
Next table shape: (17783, 7)
Next table shape: (21095, 7)
Next table shape: (20246, 7)
Next table shape: (36, 7)
Next table shape: (18437, 7)
Next table shape: (20392, 7)
Next table shape: (350, 7)
Next table shape: (5814, 7)
Next table shape: (53, 7)
Next table shape: (70, 7)
Next table shape: (15631, 7)
Next table shape: (6368, 7)
Next table shape: (12109, 7)
Next table shape: (20141, 7)
Next table shape: (188, 7)
Next table shape: (5427, 7)
Next table shape: (19, 7)
Next table shape: (344, 7)
Next table shape: (53, 7)
Next table shape: (19879, 7)
Next table shape: (13684, 7)
Next table shape: (187, 7)
Next table shape: (624, 7)
Next table shape: (415, 7)
Next table shape: (4378, 7)
Next table shape: (4996, 7)
Next table shape: (10510, 7)
Next table shape: (20028, 7)
Next table shape: (198, 7

,post_id,text,date,likes,owner_id,is_video,by_tag
0,3-7kHpM46-,"Доброе утро, друзья!! Отхватила вчера по дорог...",1.434443e+09,46,1.132830e+07,False,акварельмк
1,3-9jrIxI65,"@damarkelova ・・・\nДоброе утро, друзья!! Отхват...",1.434444e+09,199,1.101804e+09,False,акварельмк
2,3_r5r1s4yK,"Миллион оттенков розового и я @damarkelova, жд...",1.434469e+09,32,1.132830e+07,False,акварельмк
3,3_xPGMRI30,@damarkelova ・・・ Миллион оттенков розового и я...,1.434471e+09,140,1.101804e+09,False,акварельмк
4,3vos8XRI98,"Вечером в среду, после обеда...\nЖдем вас на #...",1.433930e+09,152,1.101804e+09,False,акварельмк
5,3vxZIas4-H,#Repost @culture_club with @repostapp.\n・・・\nВ...,1.433935e+09,36,1.132830e+07,False,акварельмк
6,4D_8oGs4_p,Вчера у нас прошло первое занятие по #ботаниче...,1.434613e+09,50,1.132830e+07,False,акварельмк
7,5gqU-CM4xG,"Дорогие друзья, #срочное, #пятничное!! У нас в...",1.437723e+09,55,1.132830e+07,False,акварельмк
8,5grEGfRIw_,"@damarkelova Дорогие друзья, #срочное, #пятнич...",1.437723e+09,150,1.101804e+09,False,акварельмк
9,6ey5mMh57E,"Как и обещали, с радостью представляем вам нов...",1.439807e+09,9,9.591155e+08,False,акварельмк


Pipeline

In [ ]:
def add_contacts(df):
    """Adds phone numbers and bollean direct features"""
    
    df = df.copy()
    contacts_pat = re.compile("([\+7|7|8]?[\s\-]?\(?[489][0-9]{2}\)?[\s\-]?[0-9]{3}[\s\-]?[0-9]{2}[\s\-]?[0-9]{2})|(директ)")
    df[["phone_number", "direct"]] = df["text"].str.extract(contacts_pat)
    return df

    
def add_price(df):
    """Adds price feature"""
    
    df = df.copy()
    price_pat = re.compile("([\d]+[0]{1})\s?[р]{1}")  # "420 р" and "4000р" are prices, "421" is not
    df["price"] = (df["text"].str.lower()
                   .str.extract(price_pat)
                   .fillna(0)
                   .iloc[:, 0]  # convert to Series
                   .map(lambda x: 0 if isinstance(x, str) and len(x) > 5  # remove prices more than 99999 to escape overflow
                        else x)
                   .astype("int64"))
    return df

    
def filter_workshops(df):
    """Remove all rows that are not workshops"""
    
    contact_filter = df[["phone_number", "direct"]].notnull().any(axis=1)
    price_filter = df["price"].between(500, 10000)
    return df[contact_filter & price_filter]

In [ ]:
%%time

main_data = (data.pipe(add_contacts)
                .pipe(add_price)
                .pipe(filter_workshops))

main_data.reset_index(drop=True, inplace=True)

In [ ]:
print(main_data.shape)

main_data.head()

In [ ]:
dump_path = "./data/"
main_data.to_csv(os.path.join(dump_path, "main_data.csv"), sep=";", encoding="utf-8", index=False)

#### Случайная выборка из постов для ручной проверки

In [ ]:
pd.Series(np.random.choice(test_data["post_id"], size=100)).to_csv("./data/main_random_posts.csv")